In [ ]:
import numpy as np
import pandas as pd

import sys
sys.path.append('/home/smatthe2/GillesPy2')
import gillespy2
from gillespy2 import Model, Species, Parameter, Reaction, Event, \
                      EventTrigger, EventAssignment
from gillespy2 import ODECSolver, ODESolver, SSACSolver

import matplotlib.pyplot as plt

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# devils Parameter Approximation

This notebook is used to calibrate a initial values which approximately match the results given in Cunningham, Calum X., et al. "Quantifying 25 years of disease‐caused declines in Tasmanian devil populations: host density drives spatial pathogen spread." Ecology Letters 24.5 (2021): 958-969. The objective of this notebook is to gain an "eyeball approximation" of reasonable starting values to serve as input for more sophisticated inferencing.

## Read in observed data

In [ ]:
# pop_data = pd.read_csv('../year_data/devil_data_v2.csv')
pop_data = pd.read_csv('../month_data/Devils_Dataset__Population_1985-2020.csv')

devil_pop = np.array(pop_data['Population'].iloc[:].values)

obs = np.vstack([devil_pop]).reshape(1, 1, -1)
print(obs)

In [ ]:
plt.plot(range(0,len(devil_pop)), devil_pop)

## Model

In [ ]:
class Devilsv101(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="devils_v1.0")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="r", expression="1"))
        self.add_parameter(Parameter(name="K", expression=max(devil_pop)*1.16))
        self.add_parameter(Parameter(name="ds", expression="0.22"))
        self.add_parameter(Parameter(name="dI", expression="1.28"))
        self.add_parameter(Parameter(name="L", expression=".75"))
        self.add_parameter(Parameter(name="k0", expression="4"))

        # Variables (initial values adjusted to observed data)
        self.add_species(Species(name="S", initial_value=devil_pop[0], mode="discrete"))
        self.add_species(Species(name="E", initial_value=0, mode="discrete"))
        self.add_species(Species(name="I", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Devils", initial_value=devil_pop[0], mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="birth", reactants={}, products={'S': 1, 'Devils': 1}, propensity_function="r*(S+E+I)*(1-(S+E+I)/K)"))
        self.add_reaction(Reaction(name="death_S", reactants={'S': 1, 'Devils': 1}, products={}, propensity_function="ds*S"))
        self.add_reaction(Reaction(name="transmission", reactants={'S': 1, 'I': 1}, products={'E': 1, 'I':1}, propensity_function="k0*S*I/K"))
        self.add_reaction(Reaction(name="death_E", reactants={'Devils': 1, 'E': 1}, products={}, propensity_function="ds*E"))
        self.add_reaction(Reaction(name="latency", reactants={'E': 1}, products={'I': 1}, propensity_function="E/L"))
        self.add_reaction(Reaction(name="death_I", reactants={'I': 1, 'Devils': 1}, products={}, propensity_function="dI*I"))

        # Timespan
        self.timespan(np.arange(0, 422, 1)) # month data tspan

In [ ]:
model = Devilsv101()
print(model)

## Configure Solver Widgets

In [ ]:
solver = SSACSolver(model, variable=True)

In [ ]:
def plot(start, end, results, total_devils):
    title = f'Total Species for Time Range {start}-{end}'
    
    simulated_devils = plt.plot(range(start, end+1), total_devils, 'b', label='Simulated Total')
    s_devils = plt.plot(range(start, end+1), results['S'], 'g', alpha=.3, label='Susceptible')
    i_devils = plt.plot(range(start, end+1), results['I'], 'r', alpha=.3, label='Infected')
    e_devils = plt.plot(range(start, end+1), results['E'], 'y', alpha=.3, label='Exposed')
    expected_devils = plt.plot(range(start, end), devil_pop[start:end], '--b', label='Observed Total')
    plt.xlabel('month')
    plt.ylabel('population')
    plt.legend(loc='best')
    plt.title(title)

In [ ]:
def pre_infection(start, end, birth, s_death):
    print('=================================')
    print(f'| Time Range: {start}-{end}')
    print('| -------------------------------')
    print('| Input Parameters:')
    print(f'| \tr: {birth}')
    print(f'| \tds: {s_death}')
    print('| -------------------------------')
    print(f'| Initial Value of Devils: {round(devil_pop[start])}')
    print('=================================')
    
    model.listOfSpecies['Devils'].initial_value = round(devil_pop[start])
    variables = {
        'r':str(birth),
        'ds': str(s_death)
    }
    results = solver.run(model=model, variables=variables, t=end-start, increment=1)
    
    total_devils = np.add(np.add(results['I'], results['S']), results['E'])
    
    plot(start, end, results, total_devils)
    
    return 'TOTAL Expected: {0} - Got: {1}'.format(devil_pop[end-1], total_devils[-1])

In [ ]:
def f(L, k0, r, sTime, eTime, dI, ds):
    title = 'Total Species for Time Range {0}-{1}'.format(sTime, eTime)
    print('=================================')
    print('| Time Range: {}-{}'.format(sTime, eTime))
    print('| -------------------------------')
    print('| Input Parameters:')
    print('| \tL: {}, k0: {}'.format(L, k0))
    print('| \tr: {}'.format(r))
    print('| \tds: {}'.format(ds))
    print('| -------------------------------')
#     print('| Initial Value of I: {}'.format(dfdt_pop[sTime]))
#     print('| -------------------------------')
#     print('| Ratio of dI:r:')
#     print('| \t{}:{}'.format(str(dI), str(r)))
    print('=================================')
    
    print(sTime)
    
    print(devil_pop[sTime])
    model.listOfSpecies['I'].initial_value = round(devil_pop[sTime]*.001)  # This will be overwritten by variables later
    model.listOfSpecies['S'].initial_value = round(devil_pop[sTime]*.999)
    
#     # MAP TO NO INFECTION
#     model.listOfSpecies['I'].initial_value = 0  # This will be overwritten by variables later
#     model.listOfSpecies['S'].initial_value = round(devil_pop[sTime])
    
    model.listOfSpecies['Devils'].initial_value = round(devil_pop[sTime])
    
    results = solver.run(model=model, variables={'L':str(L), 'r':str(r), 'k0':str(k0), 'dI': str(dI), 'ds': str(ds)}, t=eTime-sTime, increment=1)
#     print(results)
#     if len(res) > 0:
#         print('>0')
#         results_list = []
#         for i in range(0, len(res)):
#             temp = gillespy2.Trajectory(data=res[i], model=model, solver_name=solver.name)
#             results_list.append(temp)

#         results = gillespy2.Results(results_list)

#     print('RESULTS')
#     print(results)
    total_devils = np.add(np.add(results['I'], results['S']), results['E'])
    print(model.listOfParameters['K'])
#     print('I: ', results['I'], '\nS: ', results['S'], '\nE: ', results['E'], '\ntotal: ', total_devils)

    simulated_devils = plt.plot(range(sTime, eTime+1), total_devils, 'b', label='Simulated Total')
    s_devils = plt.plot(range(sTime, eTime+1), results['S'], 'g', alpha=.3, label='Susceptible')
    i_devils = plt.plot(range(sTime, eTime+1), results['I'], 'r', alpha=.3,label='Infected')
    e_devils = plt.plot(range(sTime, eTime+1), results['E'], 'y', alpha=.3,label='Exposed')
    expected_devils = plt.plot(range(sTime, eTime), devil_pop[sTime:eTime], '--b', label='Observed Total')
#     print('Exposed')
#     print(results['E'])
    plt.xlabel('month')
    plt.ylabel('population')
    plt.legend(loc='best')
    plt.title(title)
    result_str = 'TOTAL Expected: {0} - Got: {1}'.format(devil_pop[eTime-1], total_devils[-1])

    return result_str